In [ ]:
!pip install -q groq sentence-transformers chromadb pypdf openpyxl gradio pandas

import os
import gradio as gr
from groq import Groq
import chromadb
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader
import pandas as pd
from typing import List, Tuple
import io

class RAGApplication:
    def __init__(self):
        self.client = None
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.chroma_client = chromadb.Client()
        self.collection = None
        self.api_key_verified = False

    def verify_api_key(self, api_key: str) -> Tuple[str, bool]:
        """Verify the Groq API key by making a test call"""
        try:
            test_client = Groq(api_key=api_key)
            # Make a minimal test call
            test_client.chat.completions.create(
                messages=[{"role": "user", "content": "test"}],
                model="llama-3.1-8b-instant",
                max_tokens=5
            )
            self.client = test_client
            self.api_key_verified = True
            return "✅ API Key verified successfully!", True
        except Exception as e:
            self.api_key_verified = False
            return f"❌ API Key verification failed: {str(e)}", False

    def extract_text_from_pdf(self, file_content: bytes) -> str:
        """Extract text from PDF file"""
        pdf_reader = PdfReader(io.BytesIO(file_content))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text

    def extract_text_from_excel(self, file_content: bytes) -> str:
        """Extract text from Excel file"""
        df = pd.read_excel(io.BytesIO(file_content))
        return df.to_string()

    def extract_text_from_txt(self, file_content: bytes) -> str:
        """Extract text from text file"""
        return file_content.decode('utf-8')

    def process_document(self, file) -> str:
        """Process uploaded document and extract text"""
        if file is None:
            return "No file uploaded"

        try:
            file_content = file if isinstance(file, bytes) else open(file.name, 'rb').read()
            file_name = file.name if hasattr(file, 'name') else "unknown"

            if file_name.endswith('.pdf'):
                text = self.extract_text_from_pdf(file_content)
            elif file_name.endswith(('.xlsx', '.xls')):
                text = self.extract_text_from_excel(file_content)
            elif file_name.endswith('.txt'):
                text = self.extract_text_from_txt(file_content)
            else:
                return "Unsupported file format. Please upload PDF, Excel, or Text files."

            return text
        except Exception as e:
            return f"Error processing document: {str(e)}"

    def create_chunks(self, text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
        """Split text into overlapping chunks"""
        words = text.split()
        chunks = []

        for i in range(0, len(words), chunk_size - overlap):
            chunk = ' '.join(words[i:i + chunk_size])
            chunks.append(chunk)

        return chunks

    def upload_and_process(self, file) -> str:
        """Upload and process document, create embeddings and store in vector DB"""
        if not self.api_key_verified:
            return "⚠️ Please verify your API key first!"

        if file is None:
            return "Please upload a document"

        try:
            # Extract text
            text = self.process_document(file)
            if text.startswith("Error") or text.startswith("Unsupported"):
                return text

            # Create chunks
            chunks = self.create_chunks(text)

            # Create embeddings
            embeddings = self.embedding_model.encode(chunks)

            # Create or reset collection
            try:
                self.chroma_client.delete_collection("documents")
            except:
                pass

            self.collection = self.chroma_client.create_collection("documents")

            # Store in ChromaDB
            self.collection.add(
                embeddings=embeddings.tolist(),
                documents=chunks,
                ids=[f"chunk_{i}" for i in range(len(chunks))]
            )

            return f"✅ Document processed successfully!\n- Total chunks created: {len(chunks)}\n- Embeddings stored in vector database"

        except Exception as e:
            return f"Error: {str(e)}"

    def query_documents(self, question: str, chat_history: List) -> Tuple[str, List]:
        """Query the RAG system with a question"""
        if not self.api_key_verified:
            return "⚠️ Please verify your API key first!", chat_history

        if self.collection is None:
            return "⚠️ Please upload and process a document first!", chat_history

        try:
            # Create embedding for the question
            question_embedding = self.embedding_model.encode([question])

            # Retrieve relevant chunks
            results = self.collection.query(
                query_embeddings=question_embedding.tolist(),
                n_results=3
            )

            context = "\n\n".join(results['documents'][0])

            # Create prompt for Groq
            prompt = f"""Based on the following context from the document, answer the question.
If the answer cannot be found in the context, say "I cannot find this information in the uploaded document."

Context:
{context}

Question: {question}

Answer:"""

            # Get response from Groq
            response = self.client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model="llama-3.1-8b-instant",
                max_tokens=1024,
                temperature=0.3
            )

            answer = response.choices[0].message.content

            # Update chat history
            chat_history.append((question, answer))

            return "", chat_history

        except Exception as e:
            return f"Error: {str(e)}", chat_history

# Initialize the RAG application
rag_app = RAGApplication()

# Create Gradio interface
with gr.Blocks(title="RAG Application with Groq") as demo:
    gr.Markdown("# 📚 RAG Application with Groq API")
    gr.Markdown("Upload documents (PDF, Excel, Text) and ask questions based on their content")

    with gr.Row():
        with gr.Column(scale=2):
            api_key_input = gr.Textbox(
                label="Groq API Key",
                type="password",
                placeholder="Enter your Groq API key"
            )
        with gr.Column(scale=1):
            verify_btn = gr.Button("Verify API Key", variant="primary")

    api_status = gr.Textbox(label="API Key Status", interactive=False)

    gr.Markdown("---")

    with gr.Row():
        file_upload = gr.File(
            label="Upload Document (PDF, Excel, or Text)",
            file_types=[".pdf", ".xlsx", ".xls", ".txt"]
        )

    upload_btn = gr.Button("Process Document", variant="primary")
    upload_status = gr.Textbox(label="Processing Status", interactive=False)

    gr.Markdown("---")
    gr.Markdown("## 💬 Ask Questions")

    chatbot = gr.Chatbot(label="Conversation", height=400)
    question_input = gr.Textbox(
        label="Your Question",
        placeholder="Ask a question about the uploaded document..."
    )

    with gr.Row():
        submit_btn = gr.Button("Ask", variant="primary")
        clear_btn = gr.Button("Clear Chat")

    # Event handlers
    verify_btn.click(
        fn=rag_app.verify_api_key,
        inputs=[api_key_input],
        outputs=[api_status, gr.State()]
    )

    upload_btn.click(
        fn=rag_app.upload_and_process,
        inputs=[file_upload],
        outputs=[upload_status]
    )

    submit_btn.click(
        fn=rag_app.query_documents,
        inputs=[question_input, chatbot],
        outputs=[question_input, chatbot]
    )

    clear_btn.click(
        fn=lambda: ([], ""),
        outputs=[chatbot, question_input]
    )

    question_input.submit(
        fn=rag_app.query_documents,
        inputs=[question_input, chatbot],
        outputs=[question_input, chatbot]
    )

# Launch the app
demo.launch(debug=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 14.6 MB/s e

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-151668879.py:211: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", height=400)
/tmp/ipython-input-151668879.py:211: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(label="Conversation", height=400)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://279c96260cede40834.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
